In [2]:
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import matplotlib.axes as axe
import pandas as pd
import datetime as dt
import gurobipy as gp
from gurobipy import GRB
import cvxpy as cp

import random
from itertools import chain, combinations, tee
import time


# General CBCP Equilibrium Solver

## (Special Case) Quartic Polynomial Latency Functions

In [2]:
# x_temp = np.array([0, 1, 2, 3, 4])
# x_temp[-2:]

In [3]:
# arr = np.array([[1, 2, 3, 4, 5], [6, 7, 8, 9, 10], [11, 12, 13, 14, 15]])
# # num_cols = arr.shape[1]
# num_cols = 3
# # arr[:, -num_cols:] - arr[:, -num_cols-1:-1]

# np.linalg.norm(arr, axis=0)


In [3]:
# grad = np.array([0, 1, 2, 3, 4])
grad = np.array([2, 4, 0, 1, 3])

for id_temp, entry_temp in enumerate(grad):
    print("id_temp, entry_temp:", id_temp, entry_temp)


id_temp, entry_temp: 0 2
id_temp, entry_temp: 1 4
id_temp, entry_temp: 2 0
id_temp, entry_temp: 3 1
id_temp, entry_temp: 4 3


In [5]:
def welfare_obj(lambda_E, lambda_R, lambda_I, tau, v_I_array, v_E_array, y_el, y_in, \
                a_ex = np.array([0.0, 0.0, 0.0, 0.0, 1.0]), \
                a_gp = np.array([0.0, 0.0, 0.0, 0.0, 1.0])):
    
    # In full:
    # y_el indices: (group, edge, "lane", time)
    # y_in indices: (group, edge, "lane", time)
    # Here:
    # y_el indices: (group, "lane")
    # y_in indices: (group, "lane")
    
    num_el = v_E_array.shape[0]
    num_in = v_I_array.shape[0]
    
    ## Compute lane flows:
    
    # Express lane (ex):
    x_ex = 0.0
    x_ex += sum([y_el(g, ell) for g in range(num_el) for ell in [0, 1]])
    x_ex += sum([y_in(g, 0) for g in range(num_in)])
    
    # General purpose lane (gp):
    x_gp = 0.0 
    x_gp += sum([y_el(g, 2) for g in range(num_el)])
    x_gp += sum([y_in(g, 1) for g in range(num_in)])
    
    ell_ex = a_ex[4] * (x_ex ** 4) + a_ex[3] * (x_ex ** 3) + a_ex[2] * (x_ex ** 2) \
                + a_ex[1] * x_ex + a_ex[0]
    ell_gp = a_gp[4] * (x_gp ** 4) + a_gp[3] * (x_gp ** 3) + a_gp[2] * (x_gp ** 2) \
                + a_gp[1] * x_gp + a_gp[0]
        
    obj_E = tau * sum([y_el(g, 1) for g in range(num_el)]) \
                + sum([y_el(g, 1) * v_E_array[g] for g in range(num_el)]) * ell_ex \
                + sum([y_el(g, 2) * v_E_array[g] for g in range(num_el)]) * ell_gp
    obj_R = tau * sum([y_el(g, 1) for g in range(num_el)]) \
                + tau * sum([y_el(g, 1) for g in range(num_in)])
    obj_I = tau * sum([y_in(g, 0) for g in range(num_in)]) \
                + sum([y_in(g, 0) * v_I_array[g] for g in range(num_in)]) * ell_ex \
                + sum([y_in(g, 1) * v_I_array[g] for g in range(num_in)]) * ell_gp    

#     print("ell_ex:", ell_ex)
#     print("ell_gp:", ell_gp)
#     print("obj_E:", obj_E)
#     print("obj_R:", obj_R)

#     print("tau:", tau)
#     print("c_in_ex_toll @ y_in:", c_in_ex_toll @ y_in)
#     print("v_I:", v_I)
#     print("ell_ex:", ell_ex)
#     print("c_in_ex @ y_in:", c_in_ex @ y_in)
#     print("ell_gp:", ell_gp)
#     print("c_in_gp @ y_in:", c_in_gp @ y_in)
    
#     print("obj_I:", obj_I)

    return lambda_E * obj_E - lambda_R * obj_R + lambda_I * obj_I

    

In [ ]:
def proj_tau_B_11(tau, B, tau_max = 1.0, B_max = 1.0):
    
    tau_feas = cp.Variable(1)
    B_feas = cp.Variable(1)
    
    func = (tau_feas - tau)**2 + (B_feas - B)**2

    objective = cp.Minimize(func)
    
    constraints = []
#     constraints += [0 <= tau_feas <= 1]
#     constraints += [0 <= B_feas <= 1]
    constraints += [tau_feas >= 0.0]
    constraints += [B_feas >= 0.0]
    constraints += [tau_feas <= tau_max]
    constraints += [B_feas <= B_max]
    constraints += [B_feas <= tau_feas]
    
    prob = cp.Problem(objective, constraints)
    result = prob.solve()

    return np.round(tau_feas.value, decimals=4), np.round(B_feas.value, decimals=4)


# Chinmay's Algorithm:

In [ ]:

# Below: For quartic latency functions:
# Latency Function: a_4 x^4 + a_3 x^3 + a_2 x^2 + a_1 x + a_0
def solve_CBCP_direct(tau, B, v_I_array, v_E_array, \
                         a_ex = np.array([0.0, 0.0, 0.0, 0.0, 1.0]), \
                         a_gp = np.array([0.0, 0.0, 0.0, 0.0, 1.0])):
    
    # Variables:
    # y_el indices: (group, "lane")
    # y_in indices: (group, "lane")

    num_el = v_E_array.shape[0]
    num_in = v_I_array.shape[0]

    # Variables:
    y_el = {}
    for g in range(num_el):
        for ell in range(3):
            y_el[(g, ell)] = cp.Variable(1)
            
    y_in = {}
    for g in range(num_in):
        for ell in range(2):
            y_in[(g, ell)] = cp.Variable(1)
            
    x_ex = cp.Variable(1)
    x_gp = cp.Variable(1)
    
    # Objective:
    func = 0.0
    func += 1/5 * a_ex[4] * cp.power(x_ex, 5)
    func += 1/4 * a_ex[3] * cp.power(x_ex, 4)
    func += 1/3 * a_ex[2] * cp.power(x_ex, 3)
    func += 1/2 * a_ex[1] * cp.power(x_ex, 2)
    func += a_ex[0] * x_ex
    func += 1/5 * a_gp[4] * cp.power(x_gp, 5)
    func += 1/4 * a_gp[3] * cp.power(x_gp, 4)
    func += 1/3 * a_gp[2] * cp.power(x_gp, 3)
    func += 1/2 * a_gp[1] * cp.power(x_gp, 2)
    func += a_gp[0] * x_gp

    for v_E_index, v_E in enumerate(v_E_array):
        func += cp.sum(y_el[(g, 1) for g in range(num_el)]) * tau / v_E
    for v_I_index, v_I in enumerate(v_I_array):
        func += cp.sum(y_in[(g, 0) for g in range(num_in)]) * tau / v_I

    objective = cp.Minimize(func)
    
    # Constraints:
    
#     constraints = []
# #     constraints += [0 <= tau_feas <= 1]
# #     constraints += [0 <= B_feas <= 1]
#     constraints += [tau_feas >= 0.0]
#     constraints += [B_feas >= 0.0]
#     constraints += [tau_feas <= tau_max]
#     constraints += [B_feas <= B_max]
#     constraints += [B_feas <= tau_feas]

    ## TODO: Edit below to use the fact that creatively, CVXPY can handle multiple-dimensional \
    ## (e.g., 3d or 4d) arrays:

    constraints = []
#     constraints += ...
    
    # Express lane (ex):
    x_ex = 0.0
    x_ex += sum([y_el(g, ell) for g in range(num_el) for ell in [0, 1]])
    x_ex += sum([y_in(g, 0) for g in range(num_in)])
    
    # General purpose lane (gp):
    x_gp = 0.0 
    x_gp += sum([y_el(g, 2) for g in range(num_el)])
    x_gp += sum([y_in(g, 1) for g in range(num_in)])
    
    
    
    
    
    
    
    
    
    C_el = np.kron(np.eye(num_in), np.array([1, 1, 1]))
    C_in = np.kron(np.eye(num_el), np.array([1, 1]))
    c_el_ex_budget = np.array([0, 1, 0])
    
    constraints = []
    constraints += [y_el >= 0, y_in >= 0]
    constraints += [C_el @ y_el == 1, C_in @ y_in == 1]
    
    for v_E_index, v_E in enumerate(v_E_array):
        unit_g = np.zeros(len(v_E_array))
        unit_g[v_E_index] = 1
        c_el_ex_budget_g = np.kron(unit_g, c_el_ex_budget)
        constraints += [(c_el_ex_budget_g @ y_el) * tau <= B]
    
    prob = cp.Problem(objective, constraints)
    
    # The optimal objective value is returned by `prob.solve()`.
    result = prob.solve()
#     # The optimal value for x is stored in `x.value`.
#     print("In solve_CBCP_direct, y_el.value:", y_el.value)
#     print("In solve_CBCP_direct, y_in.value:", y_in.value)

    return y_el.value, y_in.value



In [ ]:
time_1 = time.time()

tau = 0.4
B = 0.3
v_I_array = np.array([1.0, 1.2, 1.4])
v_E_array = np.array([0.6, 0.8, 0.9])
a = np.array([0.0, 0.0, 0.0, 0.0, 1.0])
num_iters_max = 5000
error_bound = 1E-3
diffs_num_cols = 5
# y_init = np.array([0.0, 0.05, 0.95, 0.95, 0.05])

# lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0
lambda_E, lambda_R, lambda_I = 2.0, 0.2, 1.0
# lambda_E, lambda_R, lambda_I = 1.0, 1.5, 1.0

tau_max, B_max = 1.0, 1.0
d = 2
num_iters = 1000
tau = np.zeros(num_iters)
tau_perturbed = np.zeros(num_iters)
B = np.zeros(num_iters)
B_perturbed = np.zeros(num_iters)
tau_B_iters = np.zeros((2, num_iters))
delta = np.zeros(num_iters)
eta = np.zeros(num_iters)
eta_bar = 3.0
delta_bar = 3.0

welfare_list = []

# tau[0] = 0.8
# B[0] = 0.2

tau[0] = 0.6
B[0] = 0.4

for i in range(num_iters-1):
    
    print()
    print("Iter:", i)
    
    eta[i] = eta_bar * (i+1)**(-1/2) * d**(-1)
    delta[i] = delta_bar * (i+1)**(-1/4) * d**(-1/2)
    w_i_unnormalized = np.random.randn(2)
    w_i = w_i_unnormalized / np.linalg.norm(w_i_unnormalized)
    print("w_i:", w_i)
    tau_perturbed[i] = tau[i] + delta[i] * w_i[0]
    B_perturbed[i] = B[i] + delta[i] * w_i[1]
    
#     if tau_perturbed[i] < B_perturbed[i] or tau_perturbed[i] < 0 or B_perturbed[i] < 0:
    tau_perturbed[i], B_perturbed[i] = proj_tau_B_11(tau_perturbed[i], B_perturbed[i], \
                                                    tau_max = tau_max, B_max = B_max)
    
    print("tau[i]:", tau[i])
    print("B[i]:", B[i])
    print("tau_perturbed[i]:", tau_perturbed[i])
    print("B_perturbed[i]:", B_perturbed[i])


    y_el, y_in = solve_CBCP_direct(tau = tau[i], B = B[i],\
                                   v_I_array = v_I_array, v_E_array = v_E_array, a_ex = a, a_gp = a)
    
#     print()
#     print("y_el:", y_el)
#     print("y_in:", y_in)
#     print()

#     print()
#     print("New solve_CBCP_iter_11 call to solve_CBCP (perturbed):")
#     print()
    y_el_perturbed, y_in_perturbed = solve_CBCP_direct(tau = tau_perturbed[i], B = B_perturbed[i], \
                                                       v_I_array = v_I_array, v_E_array = v_E_array, a_ex = a, a_gp = a)
    
#     print("y_el:", y_el)
#     print("y_in:", y_in)
#     print("y_el_perturbed:", y_el_perturbed)
#     print("y_in_perturbed:", y_in_perturbed)
    
    welfare = welfare_obj(lambda_E, lambda_R, lambda_I, tau = tau[i], v_I_array = v_I_array, v_E_array = v_E_array, \
                          y_el = y_el, y_in = y_in, a_ex = a, a_gp = a)
    welfare_perturbed = welfare_obj(lambda_E, lambda_R, lambda_I, tau = tau_perturbed[i], v_I_array = v_I_array, v_E_array = v_E_array, \
                                    y_el = y_el_perturbed, y_in = y_in_perturbed, a_ex = a, a_gp = a)
    
    welfare_list.append(welfare)
    
#     def welfare_obj(lambda_E, lambda_R, lambda_I, tau, v_I, v_E, y_el, y_in, \
#                 a_ex = np.array([0.0, 0.0, 0.0, 0.0, 1.0]), \
#                 a_gp = np.array([0.0, 0.0, 0.0, 0.0, 1.0]))
    
    print("welfare:", welfare)
    print("welfare_perturbed:", welfare_perturbed)
#     print("tau[i] - eta[i] * (d/delta[i]) * w_i[0] * (welfare_perturbed - welfare):\n", tau[i] - eta[i] * (d/delta[i]) * w_i[0] * (welfare_perturbed - welfare))
    
    tau[i+1] = tau[i] - eta[i] * (d/delta[i]) * w_i[0] * (welfare_perturbed - welfare)
    B[i+1] = B[i] - eta[i] * (d/delta[i]) * w_i[1] * (welfare_perturbed - welfare)

#     if not (B[i+1] <= tau[i+1] and 0 <= tau[i+1] <= 1 and 0 <= B[i+1] <= 1):
    tau[i+1], B[i+1] = proj_tau_B_11(tau[i+1], B[i+1])
        
    if i >= diffs_num_cols + 2:
        tau_diffs = tau[i-diffs_num_cols : i-1] - tau[i-diffs_num_cols+1 : i]
        B_diffs = B[i-diffs_num_cols : i-1] - B[i-diffs_num_cols+1 : i]
        
#         print("tau[0:10]:", tau[0:10])
#         print("B[0:10]:", B[0:10])
        print("tau_diffs:", tau_diffs)
        print("B_diffs:", B_diffs)
        
        if max(np.max(np.absolute(tau_diffs)), np.max(np.absolute(B_diffs))) < error_bound:
            break

time_2 = time.time()

min_welfare = min(welfare_list)
argmin_welfare_list = welfare_list.index(min(welfare_list))
argmin_tau = tau[argmin_welfare_list]
argmin_B = B[argmin_welfare_list]

print()
print("Time:", time_2 - time_1)



In [ ]:
print("first(welfare_list):", welfare_list[0])
print("min(welfare_list):", min(welfare_list))
print("max(welfare_list):", max(welfare_list))
print("argmin_tau:", argmin_tau)
print("argmin_B:", argmin_B)

## Grid Search:

In [ ]:
# tau = 0.5
# B = 0.5
# v_I = 1.0
# v_E = 0.6
# a = np.array([0.0, 0.0, 0.0, 0.0, 1.0])

# lambda_E, lambda_R, lambda_I = 1.0, 1.5, 1.0

# y_el_point_5, y_in_point_5 = solve_CBCP_direct(tau = tau, B = B, v_I_array = np.array([v_I]), v_E_array = np.array([v_E]),\
#                                                a_ex = a, a_gp = a)

# welfare_obj_arr[tau_index][B_index] = welfare_obj(lambda_E, lambda_R, lambda_I, tau, v_I_array = np.array([v_I]), v_E_array = np.array([v_E]), \
#                                                   y_el = y_el_point_5, y_in = y_in_point_5, \
#                                                   a_ex = a, a_gp = a)


In [ ]:
time_1 = time.time()

tau = 0.4
B = 0.3
v_I_array = np.array([1.0])
v_E_array = np.array([0.6])
a = np.array([0.0, 0.0, 0.0, 0.0, 1.0])
num_iters_max = 5000
error_bound = 1E-3
diffs_num_cols = 5
y_init = np.array([0.0, 0.0, 1.0, 1.0, 0.0])
lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0
# lambda_E, lambda_R, lambda_I = 1.0, 0.2, 1.0
# lambda_E, lambda_R, lambda_I = 1.0, 1.5, 1.0


grid_size = 0.02
tau_arr = (np.arange(int(1/grid_size)) + 1) * grid_size
B_arr = (np.arange(int(1/grid_size)) + 1) * grid_size

welfare_obj_arr = np.ones((tau_arr.shape[0], B_arr.shape[0])) * 100
y_el_arr = np.zeros((tau_arr.shape[0], B_arr.shape[0], 3))
y_in_arr = np.zeros((tau_arr.shape[0], B_arr.shape[0], 2))
welfare_obj_list = []

for tau_index, tau in enumerate(tau_arr):
    for B_index, B in enumerate(B_arr):
        if B < tau:
            print("tau:", tau)
            print("B:", B)
            y_el_arr[tau_index, B_index, :], y_in_arr[tau_index, B_index, :] \
                = solve_CBCP_direct(tau = tau, B = B,\
                                   v_I_array = v_I_array, v_E_array = v_E_array, a_ex = a, a_gp = a)
            
#             def solve_CBCP_direct(tau, B, v_I, v_E, a = np.array([0.5, 1.0])):
            
            welfare_obj_arr[tau_index][B_index] = welfare_obj(lambda_E, lambda_R, lambda_I, tau, v_I_array = v_I_array, v_E_array = v_E_array, \
                                                              y_el = y_el_arr[tau_index, B_index, :], y_in = y_in_arr[tau_index, B_index, :], \
                                                              a_ex = a, a_gp = a)
        
            welfare_obj_list.append(welfare_obj_arr[tau_index][B_index])
            
            print()

time_2 = time.time()

print("Time:", time_2 - time_1)



In [ ]:
# tau = 0.5
# B = 0.4
# v_I = 1.0
# v_E = 0.6

# y_el_arr[tau_index, B_index, :], y_in_arr[tau_index, B_index, :] \
#                 = solve_CBCP_direct(tau = tau, B = B, v_I_array = np.array([v_I]), v_E_array = np.array([v_E]), a_ex = a, a_gp = a)
            
# #             def solve_CBCP_direct(tau, B, v_I, v_E, a = np.array([0.5, 1.0])):
            
# welfare_obj_arr[tau_index][B_index] = welfare_obj(lambda_E, lambda_R, lambda_I, tau, v_I_array = np.array([v_I]), v_E_array = np.array([v_E]), \
#                                                               y_el = y_el_arr[tau_index, B_index, :], y_in = y_in_arr[tau_index, B_index, :], \
#                                                               a_ex = a, a_gp = a)

In [ ]:
# welfare_obj_arr

argmin_indices_wrapped = np.where(welfare_obj_arr == min(welfare_obj_list))
argmin_indices = [argmin_indices_wrapped[0][0], argmin_indices_wrapped[1][0]]
# argmin_indices
argmin_tau = tau_arr[argmin_indices[0]]
argmin_B = B_arr[argmin_indices[1]]

print("argmin_tau:\n", np.round(argmin_tau, 2))
print("\nargmin_B:\n", np.round(argmin_B, 2))

welfare_obj_arr[argmin_indices[0], argmin_indices[1]]


In [ ]:
# lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0

# plt.imshow(welfare_obj_arr.T, vmin = min(welfare_obj_list), vmax = max(welfare_obj_list), origin='lower') 
plt.imshow(welfare_obj_arr.T, extent=[np.min(tau_arr), np.max(tau_arr), np.min(B_arr), np.max(B_arr)], \
           vmin = min(welfare_obj_list), vmax = max(welfare_obj_list), origin='lower') 

plt.colorbar() 
plt.xlabel("Toll")
plt.ylabel("Budget")
# plt.xticks(x_positions, x_labels)

## Test:

In [ ]:
# Test:

grad = np.array([3.11430535, 1.501, 1.501, 2.46858321, 1.501])

# y_el: \hat y_1 E, \tilde y_1 E, y_2 E
y_el_var = cp.Variable(3)
# y_in: y_1 I, y_2 I
y_in_var = cp.Variable(2)

objective = cp.Minimize(grad[0:3] @ y_el_var + grad[3:] @ y_in_var)

constraints = []
constraints += [y_el_var >= 0, y_in_var >= 0]
constraints += [cp.sum(y_el_var) == 1.0, cp.sum(y_in_var) == 1.0]
constraints += [y_el_var[1] * tau <= B]

prob = cp.Problem(objective, constraints)
result = prob.solve()

print("grad:", grad)
print("y_el_var.value:", y_el_var.value)
# print("y_el_var_current:", y_el_var_current)
print("y_in_var.value:", y_in_var.value)
# print("y_in_var_current:", y_in_var_current)
print()

# y_el_var_current = y_el_var_current + 2/(k+2) * (y_el_var.value - y_el_var_current)
# y_in_var_current = y_in_var_current + 2/(k+2) * (y_in_var.value - y_in_var_current)

# y_iters[0:3, k] = y_el_var_current
# y_iters[3:, k] = y_in_var_current

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()

# Solver=SCS,verbose=False

In [ ]:
# y_el_var
# y_in_var
# np.hstack((y_el_var, y_in_var))

# Scratch Work:

In [ ]:
x = cp.Variable(2)
y = cp.Variable(2)
v_fixed = np.array([0, 1])
objective = cp.Minimize(cp.sum_squares(x - y) + cp.sum_squares(x - v_fixed))
constraints = []
# for i in range(2):
#     constraints += [x[i] >= 2]
# constraints += [x[i] >=2 for i in range(2)]
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print("x.value:", x.value)
print("y.value:", y.value)
print()


# Scratch Work:

In [ ]:
# # Test:

# Variables:

# y_el: \hat y_1 E, \tilde y_1 E, y_2 E
y_elig = cp.Variable(3)
# y_in: y_1 I, y_2 I
y_inel = cp.Variable(2)


# Objective:

a = np.array([0.0, 0.0, 0.0, 0.0, 1.0])

tau = 0.5
B = 0.4

func = 0.0
func += 1/5 * a[4] * cp.power(y_elig[0] + y_elig[1] + y_inel[0], 5)
func += 1/4 * a[3] * cp.power(y_elig[0] + y_elig[1] + y_inel[0], 4)
func += 1/3 * a[2] * cp.power(y_elig[0] + y_elig[1] + y_inel[0], 3)
func += 1/2 * a[1] * cp.power(y_elig[0] + y_elig[1] + y_inel[0], 2)
func += a[0] * (y_elig[0] + y_elig[1] + y_inel[0])
func += y_inel[0] * tau / v_I + y_elig[0] * tau / v_E
func += 1/5 * a[4] * cp.power(y_elig[2] + y_inel[1], 5)
func += 1/4 * a[3] * cp.power(y_elig[2] + y_inel[1], 4)
func += 1/3 * a[2] * cp.power(y_elig[2] + y_inel[1], 3)
func += 1/2 * a[1] * cp.power(y_elig[2] + y_inel[1], 2)
func += a[0] * (y_elig[2] + y_inel[1])

objective = cp.Minimize(func)


# Constraints:

constraints = []
constraints += [y_elig >= 0, y_inel >= 0]
constraints += [cp.sum(y_elig) == 1, cp.sum(y_inel) == 1]
constraints += [y_elig[1] * tau <= B]

# Solve problem:
prob = cp.Problem(objective, constraints)
result = prob.solve()

# Print solution:
print("y_elig.value:", np.round(y_elig.value, 4) )
print("y_inel.value:", np.round(y_inel.value, 4) )
print()



# power(x, p)



In [ ]:
x1 = 1
x2 = 2
print("x1:", x1, ", x2:", x2)

## CVXPY can handle 4d arrays:

In [ ]:

I, J, K, L = 2, 3, 4, 5

# Variables:
x_test = {}
for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                x_test[(i, j, k,ell)] = cp.Variable(1)
            
# Objective:

func = 0.0
func += cp.sum([x_test[(i, j, k, ell)]**2 for i in range(I) for j in range(J) \
                for k in range(K) for ell in range(L)])
            
objective = cp.Minimize(func)

# Constraints:
constraints = []

for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                constraints += [cp.sum([x_test[(i, j, k, ell)] for i in range(I) for j in range(J) \
                                        for k in range(K) for ell in range(L) ]) == 1.0]
                constraints += [x_test[(i, j, k, ell)] >= 0.0 for i in range(I) for j in range(J) \
                                        for k in range(K) for ell in range(L)]

# Solve problem:
prob = cp.Problem(objective, constraints)
result = prob.solve()

# Print solution:
for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                print("i, j, k, ell:", i, j, k, ell)
                print("x_test[(i,j,k, ell)].value:", x_test[(i, j, k, ell)].value)
